In [8]:
from ase.visualize import view
from ase.io import read

index = 7

output_dir = f"toy_outputs/{index}"

bulk = read(f'{output_dir}/00_bulk.cif')
unit_slab = read(f'{output_dir}/01_unit_slab.cif')
struct_with_vac = read(f'{output_dir}/02_struct_with_vac.cif')
untiled_slab_primitive = read(f'{output_dir}/03_untiled_slab_primitive.cif')
final_slab_ab_tiled = read(f'{output_dir}/04_final_slab_ab_tiled.cif')
final_slab_tagged_constrained = read(f'{output_dir}/05_final_slab_tagged_constrained.cif')

true_slab = read(f'{output_dir}/true_slab.cif')


print("# of atoms in bulk: ", len(bulk))
print("# of atoms in unit_slab: ", len(unit_slab))
print("# of atoms in struct_with_vac: ", len(struct_with_vac))
print("# of atoms in untiled_slab_primitive: ", len(untiled_slab_primitive))
print("# of atoms in final_slab_ab_tiled: ", len(final_slab_ab_tiled))
print("# of atoms in final_slab_tagged_constrained: ", len(final_slab_tagged_constrained))

print("# of atoms in true_slab: ", len(true_slab))

# of atoms in bulk:  4
# of atoms in unit_slab:  8
# of atoms in struct_with_vac:  16
# of atoms in untiled_slab_primitive:  16
# of atoms in final_slab_ab_tiled:  64
# of atoms in final_slab_tagged_constrained:  64
# of atoms in true_slab:  64


In [7]:
# view(bulk, viewer='ngl')
view(unit_slab, viewer='ngl')
# view(struct_with_vac, viewer='ngl')
# view(untiled_slab_primitive, viewer='ngl')
# view(final_slab_ab_tiled, viewer='ngl')
# view(final_slab_tagged_constrained, viewer='ngl')
# view(true_slab, viewer='ngl')